In [1]:
%run Cube4.ipynb

import numpy as np
import random
import time
from numba import njit

Cube initialized

         64 65 66 67
         68 69 70 71
         72 73 74 75
         76 77 78 79
48 49 50 51  0 1 2 3  16 17 18 19  32 33 34 35
52 53 54 55  4 5 6 7  20 21 22 23  36 37 38 39
56 57 58 59  8 9 10 11  24 25 26 27  40 41 42 43
60 61 62 63  12 13 14 15  28 29 30 31  44 45 46 47
         80 81 82 83
         84 85 86 87
         88 89 90 91
         92 93 94 95


In [2]:
move_list = [
    "R", "R2", "R'",
    "L", "L2", "L'",
    "Rw", "Rw2", "Rw'",
    "Lw", "Lw2", "Lw'",
    "r", "r2", "r'",
    "l", "l2", "l'",
    "U", "U2", "U'",
    "D", "D2", "D'",
    "Uw", "Uw2", "Uw'",
    "Dw", "Dw2", "Dw'",
    "u", "u2", "u'",
    "d", "d2", "d'",
    "F", "F2", "F'",
    "B", "B2", "B'",
    "Fw", "Fw2", "Fw'",
    "Bw", "Bw2", "Bw'",
    "f", "f2", "f'",
    "b", "b2", "b'"
]
move_to_id = {move: i for i, move in enumerate(move_list)}
id_to_move = {i: move for i, move in enumerate(move_list)}

# @njit
def array_to_alg(array):
    # no -1s as moves
    return " ".join([move_list[id] for id in array if id != -1])

# @njit
def alg_to_array(alg):
    return np.array([move_to_id[move] for move in alg.split()])

# function to invert an alg
@njit
def invert_alg_array(alg_array):
    inv_array = alg_array[::-1]
    inv_array = np.concatenate((inv_array[inv_array!=-1], 3*inv_array[inv_array==-1]))
    return inv_array + 2*(1-inv_array%3)

# run once to compile
print(invert_alg_array(np.array([0, -1, -1, -1])))
print(array_to_alg(np.array(range(54))))
print(alg_to_array("R R2 R' L L2 L' Rw Rw2 Rw' Lw Lw2 Lw' r r2 r' l l2 l' U U2 U' D D2 D' Uw Uw2 Uw' Dw Dw2 Dw' u u2 u' d d2 d' F F2 F' B B2 B' Fw Fw2 Fw' Bw Bw2 Bw' f f2 f' b b2 b'"))

[ 2 -1 -1 -1]
R R2 R' L L2 L' Rw Rw2 Rw' Lw Lw2 Lw' r r2 r' l l2 l' U U2 U' D D2 D' Uw Uw2 Uw' Dw Dw2 Dw' u u2 u' d d2 d' F F2 F' B B2 B' Fw Fw2 Fw' Bw Bw2 Bw' f f2 f' b b2 b'
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53]


In [3]:
@njit
def cancel_alg_array(alg_array): 
    # cancels an alg array
    changes = True
    pointer = -1
    while changes:
        changes = False
        alg_array = np.concatenate((alg_array[alg_array != -1], alg_array[alg_array == -1]))

        for i in range(len(alg_array)-1):
            if alg_array[i] == -1:
                pointer = i
                break # we can stop here because we know that the rest of the array is -1
            if alg_array[i] == -2:
                alg_array[i] = -1 # we don't need to track this change because it is not a move
                changes = True
                continue
            if alg_array[i]//3 == alg_array[i+1]//3:
                # if it cancels fully, the new move is -2 (which we need to change to -1 later)
                # if it cancels partially, the current move is set to -1 and the next move is changed to the new move

                base_move = alg_array[i]//3*3
                total_rotation = (2 + alg_array[i]%3 + alg_array[i+1]%3)%4-1
                alg_array[i] = -1
                if total_rotation >= 0:
                    alg_array[i+1] = base_move + total_rotation
                else:
                    alg_array[i+1] = -2
                changes = True
        if alg_array[-1] == -2:
            alg_array[-1] = -1

    return alg_array[:pointer]

@njit
def cancel_alg_arrays(alg_array1, alg_array2):
    alg_array = np.concatenate((alg_array1, alg_array2, np.array([-1])))
    return cancel_alg_array(alg_array)

alg1, alg2 = np.array([ 2,8,14]), np.array([6, 2, 3])
cancel_alg_arrays(alg1, alg2)

array([ 2,  8, 14,  6,  2,  3], dtype=int64)

In [4]:
move_to_grip = np.array([
#   [  R, R2, R',  L, L2, L', Rw,Rw2,Rw', Lw,Lw2,Lw',  r, r2, r',  l, l2, l',  U, U2, U',  D, D2, D', Uw,Uw2,Uw', Dw,Dw2,Dw',  u, u2, u',  d, d2, d',  F, F2, F',  B, B2, B', Fw,Fw2,Fw', Bw,Bw2,Bw',  f, f2, f',  b, b2, b',]
    [  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 0. regrip
    [  2,  0,  3,  0,  0,  0,  2,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  4,  4,  4,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  0, 10,  0,  0,  0,  7,  0, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 1. <>
    [  0,  3,  1,  0,  0,  0,  0,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  5,  5,  5,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 11,  0,  0,  0,  0,  0, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 2. <R>
    [  1,  2,  0,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  6,  6,  5,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  0, 12,  0,  0,  0,  9,  0, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 3. <R'>
    # prevent U/Uw/D
    [  2,  0,  3,  0,  0,  0,  2,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 4. <>
    [  0,  3,  1,  0,  0,  0,  0,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 5. <R>
    [  1,  2,  0,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 6. <R'>
    # F-lock (next F move should be F'/Fw')
    [  8,  0,  9,  0,  0,  0,  8,  0,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  7,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 7. <>
    [  0,  9,  7,  0,  0,  0,  0,  9,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8,  8,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 8. <R>
    [  7,  8,  0,  0,  0,  0,  7,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  9,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 9. <R'>
    # F'-lock (next F move should be F'/Fw')
    [ 11,  0, 12,  0,  0,  0, 11,  0, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], #10. <>
    [  0, 12, 10,  0,  0,  0,  0, 12, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0, 11, 11, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], #11. <R>
    [ 10, 11,  0,  0,  0,  0, 10, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12, 12, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], #12. <R'>
])

move_to_grip = np.array([
#   [  R, R2, R',  L, L2, L', Rw,Rw2,Rw', Lw,Lw2,Lw',  r, r2, r',  l, l2, l',  U, U2, U',  D, D2, D', Uw,Uw2,Uw', Dw,Dw2,Dw',  u, u2, u',  d, d2, d',  F, F2, F',  B, B2, B', Fw,Fw2,Fw', Bw,Bw2,Bw',  f, f2, f',  b, b2, b',]
    [  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 0. regrip
    [  2,  0,  3,  0,  0,  0,  2,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  4,  4,  4,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 1. <>
    [  0,  3,  1,  0,  0,  0,  0,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  5,  5,  5,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 2. <R>
    [  1,  2,  0,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  6,  6,  6,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  3,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 3. <R'>
    # prevent U/Uw/D
    [  2,  0,  3,  0,  0,  0,  2,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 4. <>
    [  0,  3,  1,  0,  0,  0,  0,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 5. <R>
    [  1,  2,  0,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 6. <R'>
    # different start (Left thumb on D)
    [  0,  9,  8,  0,  0,  0,  0,  9,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  7,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 7. <>
    [  7,  0,  9,  0,  0,  0,  7,  0,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8,  8,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 8. <R'>
    [  9,  7,  0,  0,  0,  0,  9,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  9,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  9,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 9. <R2'>
])

move_to_grip = np.array([
#   [  R, R2, R',  L, L2, L', Rw,Rw2,Rw', Lw,Lw2,Lw',  r, r2, r',  l, l2, l',  U, U2, U',  D, D2, D', Uw,Uw2,Uw', Dw,Dw2,Dw',  u, u2, u',  d, d2, d',  F, F2, F',  B, B2, B', Fw,Fw2,Fw', Bw,Bw2,Bw',  f, f2, f',  b, b2, b',]
    [  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 0. regrip
    [  2,  0,  3,  0,  0,  0,  2,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  4,  4,  4,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 1. <>
    [  0,  3,  1,  0,  0,  0,  0,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  5,  5,  5,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 2. <R>
    [  1,  2,  0,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  6,  6,  6,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  3,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 3. <R'>
    # prevent U/Uw/D
    [  2,  0,  3,  0,  0,  0,  2,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 4. <>
    [  0,  3,  1,  0,  0,  0,  0,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 5. <R>
    [  1,  2,  0,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 6. <R'>
    # different start (Left thumb on D)
    [  0,  9,  8,  0,  0,  0,  0,  9,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  7,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 7. <>
    [  7,  0,  9,  0,  0,  0,  7,  0,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8,  8,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 8. <R'>
    [  9,  7,  0,  0,  0,  0,  9,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  9,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  9,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 9. <R2'>
    # Lw-lock (next Lw move should be Lw')
    [ 11,  0, 12,  0,  0,  0, 11,  0, 12,  0,  0,  1,  0,  0,  0,  0,  0,  0, 10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 10. <>
    [  0, 12, 10,  0,  0,  0,  0, 12, 10,  0,  0,  2,  0,  0,  0,  0,  0,  0, 11, 11, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 11. <R>
    [ 10, 11,  0,  0,  0,  0, 10, 11,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0, 12, 12, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 12. <R'>
    # Lw-lock (next Lw move should be Lw')
    [ 14,  0, 15,  0,  0,  0, 14,  0, 15,  0,  0,  1,  0,  0,  0,  0,  0,  0, 13, 13, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 13. <>
    [  0, 15, 13,  0,  0,  0,  0, 15, 13,  0,  0,  2,  0,  0,  0,  0,  0,  0, 14, 14, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 14. <R>
    [ 13, 14,  0,  0,  0,  0, 13, 14,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0, 15, 15, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 15. <R'>
])


# move_to_grip = np.array([ # parity
# #   [  R, R2, R',  L, L2, L', Rw,Rw2,Rw', Lw,Lw2,Lw',  r, r2, r',  l, l2, l',  U, U2, U',  D, D2, D', Uw,Uw2,Uw', Dw,Dw2,Dw',  u, u2, u',  d, d2, d',  F, F2, F',  B, B2, B', Fw,Fw2,Fw', Bw,Bw2,Bw',  f, f2, f',  b, b2, b',]
#     [  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 0. regrip
#     [  2,  0,  3,  0,  0,  0,  2,  4,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 1. <>
#     [  0,  3,  1,  0,  0,  0,  4,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  2,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 2. <R>
#     [  1,  2,  0,  0,  0,  0,  1,  2,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  3,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 3. <R'>
#     [  0,  0,  0,  0,  0,  0,  3,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 4. <R2/R2'>
#     # F lock
#     [  6,  0,  7,  0,  0,  0,  6,  8,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  5,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 5. <>
#     [  0,  7,  5,  0,  0,  0,  8,  7,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  6,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 6. <R>
#     [  5,  6,  0,  0,  0,  0,  5,  6,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  7,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 7. <R'>
#     [  0,  0,  0,  0,  0,  0,  7,  5,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8,  8,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 8. <R2/R2'>
#     # F' lock
#     [ 10,  0, 11,  0,  0,  0, 10, 12, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  9,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 9. <>
#     [  0, 11,  9,  0,  0,  0, 12, 11,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], #10. <R>
#     [  9, 10,  0,  0,  0,  0,  9, 10, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0, 11, 11, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], #11. <R'>
#     [  0,  0,  0,  0,  0,  0, 11,  9, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12, 12, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], #12. <R2/R2'>
    
# ])

# move_to_grip = np.array([ # parity
# #   [  R, R2, R',  L, L2, L', Rw,Rw2,Rw', Lw,Lw2,Lw',  r, r2, r',  l, l2, l',  U, U2, U',  D, D2, D', Uw,Uw2,Uw', Dw,Dw2,Dw',  u, u2, u',  d, d2, d',  F, F2, F',  B, B2, B', Fw,Fw2,Fw', Bw,Bw2,Bw',  f, f2, f',  b, b2, b',]
#     [  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 0. regrip
#     [  2,  0,  3,  0,  0,  0,  2,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 1. <>
#     [  0,  3,  1,  0,  0,  0,  4,  3,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  2,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 2. <R>
#     [  1,  2,  0,  0,  0,  0,  1,  2,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 3. <R'>
#     [  0,  0,  0,  0,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 4. <R2/R2'>
# ])

move_to_grip = np.array([ # parity
#   [  R, R2, R',  L, L2, L', Rw,Rw2,Rw', Lw,Lw2,Lw',  r, r2, r',  l, l2, l',  U, U2, U',  D, D2, D', Uw,Uw2,Uw', Dw,Dw2,Dw',  u, u2, u',  d, d2, d',  F, F2, F',  B, B2, B', Fw,Fw2,Fw', Bw,Bw2,Bw',  f, f2, f',  b, b2, b',]
    [  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 0. regrip
    [  2,  0,  3,  0,  0,  0,  2,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  0,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 1. <>
    [  0,  3,  1,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  2,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 2. <R>
    [  1,  2,  0,  0,  0,  0,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  3,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 3. <R'>
    # F lock
    [  5,  0,  6,  0,  0,  0,  5,  0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 4. <>
    [  0,  6,  4,  0,  0,  0,  0,  6,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  5,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 5. <R>
    [  4,  5,  0,  0,  0,  0,  4,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  6,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 6. <R'>
    # F' lock
    [  8,  0,  9,  0,  0,  0,  8,  0,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  7,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 7. <>
    [  0,  9,  7,  0,  0,  0,  0,  9,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8,  8,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 8. <R>
    [  7,  8,  0,  0,  0,  0,  7,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  9,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0], # 9. <R'>

])

# estimate the exponential growth of the number of algs using this table
# it's dependent on the number of non-zero elements in the table, but also on the "chance" we are in a certain state

# 1. count the number of non-zero elements in each row
non_zero_elements = np.sum(move_to_grip != 0, axis=1)[1:]
# 2. count the number of 0s, 1s, ... in the table
elements_count = np.bincount(move_to_grip.flatten())[1:]
# 3. convert to probability
elements_prob = elements_count / np.sum(elements_count)
# 4. estimate the exponential growth
exp_growth = np.sum(non_zero_elements * elements_prob)-1.5
print("Exponential Growth:",round(exp_growth,3))
n = 11
print(f"Estimated #algs at length {n}:",f"{int(exp_growth**n):,}")

Exponential Growth: 6.514
Estimated #algs at length 11: 895,867,556


In [5]:
@njit
def is_regripless(move_array, start_grips = np.array([1]), move_to_grip = move_to_grip):
    # check if the alg is regripless
    grips = start_grips
    for move in move_array:
        grips = move_to_grip[grips, move]
        # print(grips, move, move_list[move])
        if np.all(grips == 0):
            return False
    return True

alg_array = alg_to_array("Rw' F2 Rw' B2 Rw F2 Rw' B2 Rw2")
start_grips = np.array([1])
is_regripless(alg_array, start_grips)

False

In [6]:
@njit
def get_valid_moves(grip, last_move):
    init_moves = np.where(move_to_grip[grip])[0]
    return init_moves[init_moves//3 != last_move//3] # remove moves that are in the same axis as the last move

# same as above, but for a list of grips
@njit
def get_valid_moves_list(grips, last_move):
    possible_moves = np.zeros(45, dtype=np.int64)
    for grip in grips:
        possible_moves += move_to_grip[grip]
    viable_moves = np.where(possible_moves)[0]

    return viable_moves[viable_moves//3 != last_move//3]

@njit
def check_rlr(alg_array):
    # ex: R, L, l, r, M
    # max 2 moves from these in a row, and only in one direction (R L is fine, L R is not)
    # we check this in two steps: first we check if there are more than 2 moves from the same grip in a row

    if len(alg_array) < 2:
        return True
    
    for i in range(len(alg_array)-2):
        if alg_array[i]//18 == alg_array[i+1]//18 == alg_array[i+2]//18:
            return False
    
    for i in range(len(alg_array)-1):
        if alg_array[i]//18 == alg_array[i+1]//18 and (alg_array[i]<alg_array[i+1] or alg_array[i]%15>11 or alg_array[i+1]%15>11): # if same axis, force 1, and also make sure none of the moves are slice moves as they are covered by wide moves
            return False

    return True

@njit
def check_rlr2(alg_array):
    # same as above, but assumes the array has length 3, so we don't need to check that nor loop
    if alg_array[0]//15 == alg_array[1]//15 == alg_array[2]//15:
        return False

    if alg_array[1]//15 == alg_array[2]//15 and (alg_array[1]<alg_array[2] or alg_array[2]%15>11): # if same axis, force 1, and also make sure last move is not slice move as that is covered by wide moves. No need to check second last move is that is checked before in the function above
        return False
    return True

In [7]:
@njit
def gen_algs(length, start_grips = np.array([1,2,3]), move_to_grip = move_to_grip):

    algs = np.zeros((1000+8**length, length), dtype = np.int8) # the exponential factor seems to be about 10.14, so we need something bigger. This is not the case for the first few lengths, so we add 5 to be safe (this is tested to work for start_grip = 1)
    i = 0 # keep track of the number of algs we have

    current_grip = np.zeros((length+1, len(start_grips)), dtype = np.int8) # first grip is the start grip
    current_grip[0] = start_grips # this will stay unchanged
    current_alg = -np.ones(length+1, dtype = np.int8)
    current_alg[0] = -1 # not a possible move, so we can start at 0

    pointer = 1 # start pointing at the first move (we use this for both algs and grips)

    while pointer > 0:
        # criterion for adding algs: pointer == length
        if pointer == length:
            # if the alg has rlr without the first move, we don't need to check all the other moves
            if check_rlr(current_alg[1:-1]):
                # loop though and add the alg to the array if the last move doesn't regrip
                valid_moves = get_valid_moves_list(current_grip[pointer-1], current_alg[pointer-1])
                # print(valid_moves)
                for move in valid_moves:
                    current_alg[pointer] = move
                    if check_rlr2(current_alg[-3:]): # check if the last two moves are rlr
                        algs[i] = current_alg[1:]
                        i += 1
            # then we go back one step (instead of setting current_alg[pointer] to 45 and dealing with it next cycle)
            current_alg[pointer] = -1
            pointer -= 1
            continue

        # if we haven't gone back, we increment the current move, set the next grip, and increment the pointer
        current_alg[pointer] += 1
        # make sure the move is valid: 1) it's a valid move, 2) it's not on the same face as the previous move, and 3) it's a regripless move
        while True: #current_alg[pointer] < 45 and current_alg[pointer]//3 == current_alg[pointer-1]//3 and move_to_grip[current_grip[pointer-1], current_alg[pointer]] != 0:
            if current_alg[pointer] == 45:
                break
            if current_alg[pointer]//3 != current_alg[pointer-1]//3 and (move_to_grip[current_grip[pointer-1], current_alg[pointer]]).any() != 0:
                break
            current_alg[pointer] += 1
        # criterion for going back: current_alg[pointer] == 45
        if current_alg[pointer] == 45:
            current_alg[pointer] = -1
            pointer -= 1
            continue
        
        # update the grip
        for j in range(len(start_grips)):
            current_grip[pointer, j] = move_to_grip[current_grip[pointer-1, j], current_alg[pointer]]
        pointer += 1
    
    return algs[:i]

# run once to compile
N = 9
start_grips = np.array([1])
algs = gen_algs(N, start_grips)
len(algs)

6692883

In [8]:
def gen_algs_cumulative(max_length, start_grips = [1], min_length = 1): #TODO add more start-grips. Also make list unique after
    # generates algs of length min_length to max_length
    algs = []
    t = time.time()
    for i in range(min_length, max_length+1):
        new_algs = gen_algs(i, start_grips)
        # pad with max_length - i -1s
        new_algs = np.pad(new_algs, ((0,0),(0,max_length-i)), constant_values = -1)
        algs.append(new_algs)
        print(f"Algs of length {i} genned. Time spent: {time.time()-t:.2f}s")

    print(f"Done genning algs. Making into np array. Time spent: {time.time()-t:.2f}s")
    algs = np.concatenate(algs)

    return algs

N = 9
start_grips = (np.array(range(len(move_to_grip)))+1)[:-1]
algs = gen_algs_cumulative(N, start_grips)
len(algs)

Algs of length 1 genned. Time spent: 0.00s
Algs of length 2 genned. Time spent: 0.00s
Algs of length 3 genned. Time spent: 0.00s
Algs of length 4 genned. Time spent: 0.00s
Algs of length 5 genned. Time spent: 0.01s
Algs of length 6 genned. Time spent: 0.04s
Algs of length 7 genned. Time spent: 0.23s
Algs of length 8 genned. Time spent: 1.47s
Algs of length 9 genned. Time spent: 10.93s
Done genning algs. Making into np array. Time spent: 10.93s


36822223

In [9]:
for i in range(10):
    print(array_to_alg(algs[random.randint(0, len(algs))]))

R' U Rw2 U2 Rw2 F' U' R2 U
U F U Rw F' U F' Rw' R
Rw U' Rw' U2 Rw U' F Rw' U'
R' F' U2 F U' Rw2 U R' F
F' Rw' R' F R F Rw
F U2 F' U' Rw F' U' Rw' U2
Rw R U R' F Rw R2 F U
U' R2 U2 F' U' F R2 U' Rw2
U2 F R F U' Rw' U2 Rw2 U
U2 Rw U' Rw2 R U2 Rw' U2 Rw


In [11]:
'''
Now we need a 4x4 representation for what we're genning

We will 0 out the top layer and the BR edge, as well as all corners.
Each center (except for 1) will get their unique number, and the cross edges as well

We will gen algs using this cube, then sort them into what they do after
To do so, we need to generate one of each cycle, then apply the alg to the cube in reverse to see what cycle it solves
'''
def gen_table(algs, mode): # Idea: Can we use np arrays instead so we can njit this? Another idea to make it less memory intensive is to append indexes to the table instead of the algs themselves. This should not affect the speed that much as the lookup is O(1) anyway
    # we won't bother creating a Cube object, we will just use the array-based alg
    table = {}
    t = time.time()
    print(f"Genning table based on {len(algs)} algs")
    cube = Cube4(mode) # 5 for RB, 6 for RF, 7 for LL removed
    cubestate = cube.state
    for i, alg in enumerate(algs):
        if i and i%(len(algs)//100) == 0:
            print(f"{i} ({i/len(algs)*100:.2f}%) algs added in {time.time()-t:.2f} seconds")
        cubestate = _apply_int_moves(cubestate, invert_alg_array(alg))
        ID = id_from_state(cubestate)
        if ID not in table:
            table[ID] = [alg.tobytes()]
        else:
            table[ID].append(alg.tobytes())
        cubestate = _apply_int_moves(cubestate, alg)
    
    return table

mode = 7
table = gen_table(algs, mode)
len(table)

Genning table based on 36822223 algs
368222 (1.00%) algs added in 7.04 seconds
736444 (2.00%) algs added in 14.81 seconds
1104666 (3.00%) algs added in 21.93 seconds
1472888 (4.00%) algs added in 29.40 seconds
1841110 (5.00%) algs added in 36.60 seconds
2209332 (6.00%) algs added in 43.61 seconds
2577554 (7.00%) algs added in 50.89 seconds
2945776 (8.00%) algs added in 58.04 seconds
3313998 (9.00%) algs added in 65.36 seconds
3682220 (10.00%) algs added in 72.40 seconds
4050442 (11.00%) algs added in 79.92 seconds
4418664 (12.00%) algs added in 86.91 seconds
4786886 (13.00%) algs added in 94.19 seconds
5155108 (14.00%) algs added in 101.60 seconds
5523330 (15.00%) algs added in 108.60 seconds
5891552 (16.00%) algs added in 115.62 seconds
6259774 (17.00%) algs added in 123.20 seconds
6627996 (18.00%) algs added in 130.01 seconds
6996218 (19.00%) algs added in 137.42 seconds
7364440 (20.00%) algs added in 145.00 seconds
7732662 (21.00%) algs added in 152.98 seconds
8100884 (22.00%) algs 

22373935

In [12]:
@njit
def has_inner_moves(alg):
    # check if the alg has inner moves
    for move in alg:
        if move%18 > 5:
            return True
    return False

In [ ]:
def gen_wing_cycles(algs, table, mode = 6):
    # we won't bother creating a Cube object, we will just use the cubestate array
    cycle_algs = []
    t = time.time()
    print(f"Genning cycle algs based on {len(algs)} algs")

    cube = Cube4(mode) # 5 for RB, 6 for RF
    cubestate = cube.state

    for i, setup in enumerate(algs):
        if i and i%(len(algs)//100) == 0:
            # make it unique
            cycle_algs = list(set(tuple(map(tuple, cycle_algs))))
            print(f"{i} ({i/len(algs)*100:.2f}%) setups checked in {time.time()-t:.2f} seconds. {len(cycle_algs)} cycle algs found")
            
        # we don't want to start with a U as that's redundant. We have to check for D starts in case D in the end isnt fingertrickable
        if setup[0]//3 == 6:
            continue
        cubestate = _apply_int_moves(cubestate, setup)

        ID = id_from_state(cubestate)
        if ID in table:
            for finish in table[ID]:
                alg = cancel_alg_arrays(setup, finish)

                if len(alg) == 0:
                    continue
                if alg[0]//3 == 6 or alg[-1]//3 == 6: # we don't want to end with U
                    continue
                if alg[0]//3 == 7 and alg[-1]//3 == 7: # if we start and end with D*, we remove it as we only need to do it once
                    continue
                # print(array_to_alg(setup),"-", array_to_alg(finish),"-", array_to_alg(alg))
                # print(setup, finish, alg)
                if check_rlr(alg) and has_inner_moves(alg):
                    cycle_algs.append(alg)
        cubestate = _apply_int_moves(cubestate, invert_alg_array(setup))
    
    cycle_algs = list(set(tuple(map(tuple, cycle_algs))))
    return cycle_algs

# N = 7
# start_grips = np.array([1,2,3])
# algs = gen_algs_cumulative(N, start_grips)

# cycle_algs = gen_wing_cycles(algs, table)
# print(len(cycle_algs))
# i = -1

In [ ]:
i += 3
print(array_to_alg(cycle_algs[i]))

In [13]:
# convert the algs to strings
# we also check if the algs actually performs a wing cycle
@njit
def check_wings(cubestate):
    # cube in mode 2
    # check all wings, we only need to check one sticker per wing
    pairs = np.array([[1,2],[17,18],[33,34], [49,50], [4,8],[20,24],[36,40],[52,56], [13,14],[29,30],[45,46],[61,62]])
    for pair in pairs:
        if cubestate[pair[0]] != cubestate[pair[1]]:
            return False
    return True

In [ ]:
# sort by length
cycle_algs.sort(key = lambda x: len(x))

cycle_algs_str = []
for alg in cycle_algs:
    str_alg = array_to_alg(np.array(alg))
    cube = Cube4(2)
    cube.apply_int_moves(alg)
    if not check_wings(cube.state):
        cycle_algs_str.append(str_alg)

# remove duplicates
cycle_algs_str = list(set(cycle_algs_str))

# sort by length
cycle_algs_str.sort(key = lambda x: len(x.split(" ")))

In [ ]:
# save to file
with open("cycle_algs_RF_.txt", "w") as file:
    file.write("\n".join(cycle_algs_str))

# read from file
# with open("cycle_algs_RF.txt", "r") as file:
#     cycle_algs_str = file.read().split("\n")

In [ ]:
def make_sub_codes(pair_set):
    # returns a list of all subsets of the pair_set
    # we do it recursively

    if len(pair_set) == 1:
        return [pair_set[0], pair_set[0][::-1]]
    
    sub_codes = []
    for i in range(len(pair_set)):
        sub_pairs = pair_set[:i] + pair_set[i+1:]
        for sub_code in make_sub_codes(sub_pairs):
            sub_codes.append(pair_set[i]+sub_code)
            sub_codes.append(pair_set[i][::-1]+sub_code)

    # make the list unique
    sub_codes = list(set(sub_codes))
    return sub_codes


def gen_valid_codes():
    # first we make all the valid sets of numbers
    # a set contains 5 pairs of numbers, and each pair is a number from 1 to 5
    
    valid_sets = []
    # 2-cycle
    valid_sets.append(["12", "12", "33", "44", "55"])
    # 3-cycle
    valid_sets.append(["13", "12", "23", "44", "55"])
    # 4-cycle
    valid_sets.append(["14", "12", "23", "34", "55"])
    # 5-cycle
    valid_sets.append(["15", "12", "23", "34", "45"])

    # TODO: FIX THE CODE FOR MULTIPLE CYCLES
    # 2-2-cycle
    valid_sets.append(["12", "12", "34", "34", "55"])
    # 2-3-cycle
    valid_sets.append(["12", "12", "34", "45", "53"])
    # 3-2-cycle
    valid_sets.append(["13", "12", "23", "45", "45"])

    codes = []

    for valid_set in valid_sets:


        # first we add the inverted first pair, as we want to do that anyways
        start = valid_set[0][::-1]
        subcodes = make_sub_codes(valid_set[1:])
        # remove everything that doesn't have 1 in the 0 and 1 position
        subcodes = [subcode for subcode in subcodes if subcode[0] == "1" or subcode[1] == "1"]
        # remove everything where the solved pairs are not in the correct order, this can include 33 and 44, 33 and 55, 44 and 55
        valid_subcodes = []
        for subcode in subcodes:
            # slice list into pairs
            pairs = [subcode[i:i+2] for i in range(0, len(subcode), 2)]
            solved_pairs = [pair for pair in pairs if pair[0] == pair[1]]
            min_solved = 0
            for i in range(len(solved_pairs)-1):
                if solved_pairs[i] > solved_pairs[i+1]:
                    min_solved = 1
                    break
            if min_solved:
                continue
            valid_subcodes.append(subcode)
        # sort the list
        valid_subcodes.sort()
        print("br",len(valid_subcodes))

        for subcode in valid_subcodes:
            codes.append(start+subcode)


        if "55" in valid_set:
            # put 55 as the first pair and the next as the first pair
            start = "55" + valid_set[0]
            # remove the first (12 or similar) and last pair (55)
            subcodes = make_sub_codes(valid_set[1:-1])
            # remove all the ones that containt 44 that is not in the last spot
            valid_subsets = []
            for subcode in subcodes:
                pairs = [subcode[i:i+2] for i in range(0, len(subcode), 2)]
                if "44" in pairs and pairs[-1] != "44":
                    continue
                valid_subsets.append(subcode)
            valid_subsets.sort()
            print("top",len(valid_subsets))
            for subcode in valid_subsets:
                codes.append(start+subcode)
            
    return codes

    

codes = gen_valid_codes()
print(codes)
len(codes)

In [ ]:
# @njit
def code_to_cubestate(code, mode):
    cube = Cube4(mode)
    state = cube.state

    if mode == 5:
        index_pairs = np.array([[27,40],[23,36],[1,77],[2,78],[17,75],[18,71],[33,66],[34,65],[49,68],[50,72]])
    if mode == 6:
        index_pairs = np.array([[11,24],[7,20],[1,77],[2,78],[17,75],[18,71],[33,66],[34,65],[49,68],[50,72]])


    for i, pair in enumerate(code):
        pair = int(pair)
        state[index_pairs[i][0]] = pair
        state[index_pairs[i][1]] = pair

    return state

def cube_to_code(cube):

    index_pairs = [27,23,1,2,17,18,33,34,49,50]

    code = ""
    for i in range(index_pairs):
        code += str(cube.state[index_pairs[i]])
    return code

In [ ]:
def case_table(algs, cases, mode):
    t = time.time()
    # first make a table
    table = {}
    for case in cases:
        table[case] = []
    table["0"] = []

    # then fill it
    cube = Cube4(6)
       
    for i, alg in enumerate(algs):
        if i and i%(len(algs)//100) == 0:
            print(f"{i} ({i/len(algs)*100:.2f}%) algs added in {time.time()-t:.2f} seconds")
        
        added = False
        for u in ["","U ","U2 ","U' "]:
            for case in cases:
                cube.state = code_to_cubestate(case, mode)
                cube.apply_moves(u+alg)
                if check_wings(cube.state):
                    table[case].append(u+alg)
                    added = True
                    break
            if added:
                break
        if not added:
            table["0"].append(alg)

    return table


mode = 6
cycle_table = case_table(cycle_algs_str, codes, mode)

In [20]:
# import json

# with open("_.json", "w") as file:
#     json.dump(cycle_table, file, indent=4)

# get random key from table
key = list(table.keys())[0]
decompressed_array = np.frombuffer(table[key][0], dtype=np.int8)

array([ 0, -1, -1, -1, -1, -1, -1, -1, -1], dtype=int8)

In [22]:
# gen parity algs

def gen_LL_algs(algs, table):
    # we won't bother creating a Cube object, we will just use the cubestate array
    LL_algs = []
    t = time.time()
    print(f"Genning LL algs based on {len(algs)} algs")

    cube = Cube4(7) # 7 for LL removed
    cubestate = cube.state

    for i, setup in enumerate(algs):
        if i and i%(len(algs)//100) == 0:
            # make it unique
            LL_algs = list(set(tuple(map(tuple, LL_algs))))
            print(f"{i} ({i/len(algs)*100:.2f}%) setups checked in {time.time()-t:.2f} seconds. {len(LL_algs)} LL algs found")
            
        # we don't want to start with a U as that's redundant. We have to check for D starts in case D in the end isnt fingertrickable
        if setup[0]//3 == 6:
            continue
        cubestate = _apply_int_moves(cubestate, setup)

        ID = id_from_state(cubestate)
        if ID in table:
            for finish in table[ID]:
                alg = cancel_alg_arrays(setup, np.frombuffer(finish, dtype=np.int8))

                if len(alg) == 0:
                    continue
                if has_inner_moves(alg):
                    LL_algs.append(alg)
        cubestate = _apply_int_moves(cubestate, invert_alg_array(setup))
    
    LL_algs = list(set(tuple(map(tuple, LL_algs))))
    print(f"Done genning LL algs. Time spent: {time.time()-t:.2f}s. {len(LL_algs)} LL algs found")
    return LL_algs

LL_algs = gen_LL_algs(algs, table)

Genning LL algs based on 36822223 algs
368222 (1.00%) setups checked in 14.78 seconds. 182913 LL algs found
736444 (2.00%) setups checked in 24.65 seconds. 272323 LL algs found
1104666 (3.00%) setups checked in 32.64 seconds. 341309 LL algs found
1472888 (4.00%) setups checked in 46.98 seconds. 446354 LL algs found
1841110 (5.00%) setups checked in 60.21 seconds. 511701 LL algs found
2209332 (6.00%) setups checked in 75.51 seconds. 636157 LL algs found
2577554 (7.00%) setups checked in 89.82 seconds. 736230 LL algs found
2945776 (8.00%) setups checked in 103.17 seconds. 795492 LL algs found
3313998 (9.00%) setups checked in 117.23 seconds. 876254 LL algs found
3682220 (10.00%) setups checked in 128.62 seconds. 957953 LL algs found
4050442 (11.00%) setups checked in 129.87 seconds. 957953 LL algs found
4418664 (12.00%) setups checked in 131.40 seconds. 957953 LL algs found
4786886 (13.00%) setups checked in 133.59 seconds. 957953 LL algs found
5155108 (14.00%) setups checked in 136.00 s

In [23]:
# check if the algs actually does parity

@njit
def check_parity(cubestate): # requires F2L
    # first, check that all edges are fixed
    pairs = np.array([[1,2],[77,78],[17,18],[71,75],[33,34],[65,66],[49,50],[68,72]])
    for pair in pairs:
        if cubestate[pair[0]] != cubestate[pair[1]]:
            return False
    # then check if we have parity or not
    s = 0
    for i in np.array([1,17,33,49]):
        s += (cubestate[i] == 5)

    return s % 2 == 1

t = time.time()
parity_algs = []
for i, alg in enumerate(LL_algs):
    if i and i%(len(LL_algs)//100) == 0:
        print(f"{i} ({i/len(LL_algs)*100:.2f}%) algs checked in {time.time()-t:.2f} seconds. {len(parity_algs)} parity algs found")
    cube = Cube4()
    cube.apply_int_moves(alg)
    if check_parity(cube.state):
        parity_algs.append(array_to_alg(alg))
        # print(f"{array_to_alg(alg)} ({len(alg)})")

28762 (1.00%) algs checked in 2.50 seconds. 0 parity algs found
57524 (2.00%) algs checked in 2.99 seconds. 0 parity algs found
86286 (3.00%) algs checked in 3.47 seconds. 0 parity algs found
115048 (4.00%) algs checked in 3.94 seconds. 0 parity algs found
143810 (5.00%) algs checked in 4.30 seconds. 0 parity algs found
172572 (6.00%) algs checked in 4.73 seconds. 0 parity algs found
201334 (7.00%) algs checked in 5.37 seconds. 0 parity algs found
230096 (8.00%) algs checked in 5.84 seconds. 0 parity algs found
258858 (9.00%) algs checked in 6.39 seconds. 0 parity algs found
287620 (10.00%) algs checked in 6.91 seconds. 0 parity algs found
316382 (11.00%) algs checked in 7.53 seconds. 0 parity algs found
345144 (12.00%) algs checked in 8.09 seconds. 0 parity algs found
373906 (13.00%) algs checked in 8.59 seconds. 0 parity algs found
402668 (14.00%) algs checked in 9.00 seconds. 0 parity algs found
431430 (15.00%) algs checked in 9.46 seconds. 0 parity algs found
460192 (16.00%) algs c

In [26]:
parity_algs

["F' Rw U2 Rw U2 Rw2 U2 Rw F2 Rw2 U2 Rw U2 Rw' U2 Rw2 F'",
 "Rw' U2 Rw' U2 Rw R' F R F' Rw2 U' R2 U2 R2 U Rw' U2 Rw'"]

In [29]:
cube = Cube4()
solved_states = []
for i in range(4):
    cube.state = cube.state
    solved_states.append(cube.state.copy())
    cube.apply_moves("U")

In [77]:
@njit
def check_pll_parity(cubestate):
    corners = [0,16,32,48]
    edges = [1,17,33,49]
    swaps = 0
    # we need to determine number of swaps in the corners and edges, then add them together mod 2
    corners_visited = []
    



    return swaps % 2

cube = Cube4()
pll_parity = "r2 U2 r2 Uw2 r2 u2"
cube.apply_moves(pll_parity)
cube.apply_moves("R U' R U R U R U' R' U' R2")
print(cube)

check_pll_parity(cube.state)


         5 5 5 5
         5 5 5 5
         5 5 5 5
         5 5 5 5
4 2 2 4  1 4 4 1  2 3 3 2  3 1 1 3
4 4 4 4  1 1 1 1  2 2 2 2  3 3 3 3
4 4 4 4  1 1 1 1  2 2 2 2  3 3 3 3
4 4 4 4  1 1 1 1  2 2 2 2  3 3 3 3
         6 6 6 6
         6 6 6 6
         6 6 6 6
         6 6 6 6
False True
parity: 1
False True
parity: 2
e
False True
parity: 4
e
False True
parity: 6


0

In [31]:
@njit
def check_pll(cubestate):
    ids = [64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79]
    for id in ids:
        if cubestate[id] != 5:
            return False
    return True

def check_solved(cubestate):
    for state in solved_states:
        if np.all(cubestate == state):
            return True
    return False



for alg in LL_algs:
    cube = Cube4()
    cube.apply_int_moves(alg)
    if check_pll(cube.state) and not check_solved(cube.state) and check_wings(cube.state):
        print(array_to_alg(alg))

R2 U2 Rw' R' U2 R2 U2 Rw2 R Rw' R2 U' Rw2 U2 R2 U2 Rw2 U
R' U2 R U F U F' Rw' R U2 R' U2 Rw U2 F U' F' U'
F' U' R F R' U2 Rw2 R2 Rw2 R2 U R F' R' U2 F U
R2 U2 R2 F Rw2 U' R2 F' R2 U2 R2 F R2 U' Rw2 F' U
Rw2 U2 Rw' R U2 Rw2 U2 R2 Rw2 R2 U2 Rw' R U2 Rw2 U2
R2 U2 R2 U2 R2 U' Rw' R Rw R U2 R2 U2 R2 U2
F' U Rw' U R U F U' F' R' U' Rw U' F R' U' R U2
Rw' U2 R' U2 Rw U2 R' F' U F R U2 R' F' U' F R2 U2
R U2 F' Rw' F U' F' R F U' R' U2 R F' Rw R' F R'
R U2 R' F' U' F Rw R' Rw' R U' F' U2 F R U R'
Rw U2 R' F U' F' U Rw2 R' Rw2 U' F U F' R2 U2 Rw'
Rw2 U2 R2 U2 Rw2 U Rw R' U2 Rw2 U2 R2 U2 Rw2 U2 Rw' R U
F Rw F' Rw' F U F' Rw F U2 R' U R U Rw' F' U
F R U Rw2 F U' F' U R U' F U F' Rw2 R' U' R' F'
R2 F R U2 Rw' R U F' U2 F U Rw U2 R2 F' R2 U
Rw2 F' U' R' U' R U F2 U' R' U R U F' Rw2 U'
Rw' U Rw U2 R' F' R' F R U2 R' F' R F Rw' R U' Rw
Rw' F U F' U' R U R2 F U' F' U R2 U2 R' U Rw U
F R' U' R2 U' R2 U2 Rw R Rw' U' F' R' U' F' U F R
R2 U2 Rw' R' U2 R2 U2 Rw2 R2 Rw' R U R2 U2 R2 U2 R2
R U F' R' F U Rw' R

In [ ]:
t = "Rw' U Rw' U' Rw2 R' U' Rw' U'"

cube = Cube4(7)
cube.apply_moves(t)

ID = id_from_state(cube.state)
print(ID)

print(array_to_alg(table[ID][0]))



In [ ]:
inv = inverse_alg("R U2 Rw' U' Rw' U2 Rw' U2 Rw'")
cube = Cube4(7)
cube.apply_moves(inv)
ID = id_from_state(cube.state)
print(ID)
print(array_to_alg(table[ID][0]))

In [ ]:
cube = Cube4(7)
cube.apply_moves(t)
print(cube)
cube.apply_moves(inverse_alg(inv))
print(cube)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
